<a href="https://colab.research.google.com/github/fabriciobotelho99/odyssey-project/blob/main/processing%20mamoas/training_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/yolov5
%pip install -qr requirements.txt # install dependencies

/content/drive/MyDrive/yolov5
     |████████████████████████████████| 596 kB 10.3 MB/s 


In [ ]:
import torch
import os
from IPython.display import Image, clear_output  # to display images
import glob
import cv2
import os.path
from pathlib import Path
import shutil
from os.path import exists
import yaml

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0+cu113 (Tesla K80)


In [ ]:
n_epochs = 200
batch_size = 16
IoU_t = 0.2


In [ ]:
def createYOLOYaml(t_dir, v_dir, n_classes):
  # Create .yaml file 

  mamoas = dict(
      train = t_dir,
      val = v_dir,
      nc = n_classes,
      names = ['mamoa']
  )
  with open('/content/drive/MyDrive/yolov5/data/mamoas.yaml', 'w') as f:
      yaml.dump(mamoas, f, default_flow_style=True)

In [ ]:
def applyGroundTruth(src_annotations, src_images, out_path):

  # read images 
  resolution_of_img = 640

  for f in glob.glob(src_images + "*.jpg"):
    img = cv2.imread(f)

    content = open(src_annotations + str(Path(f).stem) + '.txt', 'r')
    
    # reading the file
    data = content.read()
    data_into_list = data.replace('\n', '').split(' ')
    data_into_list.pop(0)

    i = 0
    while data_into_list:
      x_center = float(data_into_list[0]) * resolution_of_img
      data_into_list.pop(0)
      y_center = float(data_into_list[0]) * resolution_of_img
      data_into_list.pop(0)
      width = float(data_into_list[0]) * resolution_of_img
      data_into_list.pop(0)
      height = float(data_into_list[0]) * resolution_of_img
      data_into_list.pop(0)

      #print(x_center, y_center, width, height)
      img = cv2.circle(img, (int(x_center), int(y_center)), radius = 2, color = (0, 255, 0), thickness = -1)
      xmin = x_center - width/2
      xmax = x_center + width/2
      ymin = y_center - height/2
      ymax = y_center + height/2
      img = cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 1)
      #print(xmin, xmax, ymin, ymax)
      i = i + 1

    cv2.imwrite(out_path + str(Path(f).stem) + ".jpg", img)

    content.close()

In [ ]:
# train

n_dims = 6
data = ["/content/drive/MyDrive/yolov5/data/mamoas1.yaml", 
        "/content/drive/MyDrive/yolov5/data/mamoas2.yaml",
        "/content/drive/MyDrive/yolov5/data/mamoas3.yaml",
        "/content/drive/MyDrive/yolov5/data/mamoas4.yaml",
        "/content/drive/MyDrive/yolov5/data/mamoas5.yaml",
        "/content/drive/MyDrive/yolov5/data/mamoas6.yaml"]

for i in range(n_dims):
  !python train.py --proj /content/drive/MyDrive/yolov5/results --img 640 --batch {batch_size} --epochs {n_epochs} --data {data[i]} --weights yolov5s.pt --cache 

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/yolov5/data/mamoas5.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/yolov5/results/trainFolds, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, io

In [ ]:
# evaluate test dataset
weights = ["/content/drive/MyDrive/yolov5/results/exp/weights/best.pt",
           "/content/drive/MyDrive/yolov5/results/exp2/weights/best.pt",
           "/content/drive/MyDrive/yolov5/results/exp3/weights/best.pt",
           "/content/drive/MyDrive/yolov5/results/exp4/weights/best.pt",
           "/content/drive/MyDrive/yolov5/results/exp5/weights/best.pt",
           "/content/drive/MyDrive/yolov5/results/exp6/weights/best.pt"]
for i in range(n_dims):
  !python val.py --proj /content/drive/MyDrive/yolov5/results/val --weights {weights[i]} --data {data[i]} --img 640


val: data=/content/drive/MyDrive/yolov5/data/mamoas1.yaml, weights=['/content/drive/MyDrive/yolov5/results/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/yolov5/results/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
100% 755k/755k [00:00<00:00, 39.0MB/s]
val: Scanning '/content/drive/MyDrive/dataset_v3/test/data_15_15.cache' images and labels... 16 found, 0 missing, 0 empty, 0 corrupt: 100% 16/16 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 1/1 [00:13<00:00, 13.83s/it]
                 all         16         30      0.724        0.7      0.705 

In [ ]:
confidence = [0.408,0.215,0.216,0.064,0.176,0.420]
sources = ["/content/drive/MyDrive/dataset_v3/test/data_15_15/",
           "/content/drive/MyDrive/dataset_v3/test/data_20_20/",
           "/content/drive/MyDrive/dataset_v3/test/data_22_22/",
           "/content/drive/MyDrive/dataset_v3/test/data_25_25/",
           "/content/drive/MyDrive/dataset_v3/test/data_30_30/",
           "/content/drive/MyDrive/dataset_v3/test/data_35_35/"]

for i in range(n_dims):
  !python detect.py --proj /content/drive/MyDrive/yolov5/results/detect --conf {confidence[i]} --weights {weights[i]} --source {sources[i]} --save-txt --save-conf

detect: weights=['/content/drive/MyDrive/yolov5/results/exp/weights/best.pt'], source=/content/drive/MyDrive/dataset_v3/test/data_15_15/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.408, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/yolov5/results/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-157-g7043872 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/16 /content/drive/.shortcut-targets-by-id/1JRRXLFc6696iM81rVjN9Y43IlEVwyRSY/dataset_v3/test/data_15_15/21_1.jpg: 640x640 1 mamoa, Done. (0.013s)
image 2/16 /content/drive/.shortcut-targets-by-id/1JRRXLFc6696iM81rVjN9Y43IlEVwyRSY/dataset_v3/test/data_15_15/29_2.jpg: 640x640 5 

In [ ]:
src_annotations = ["/content/drive/MyDrive/dataset_v3/test/data_15_15/",
                   "/content/drive/MyDrive/dataset_v3/test/data_20_20/",
                   "/content/drive/MyDrive/dataset_v3/test/data_22_22/",
                   "/content/drive/MyDrive/dataset_v3/test/data_25_25/",
                   "/content/drive/MyDrive/dataset_v3/test/data_30_30/",
                   "/content/drive/MyDrive/dataset_v3/test/data_35_25/"]         


src_images = ["/content/drive/MyDrive/yolov5/results/detect/exp/",
              "/content/drive/MyDrive/yolov5/results/detect/exp2/",
              "/content/drive/MyDrive/yolov5/results/detect/exp3/",
              "/content/drive/MyDrive/yolov5/results/detect/exp4/",
              "/content/drive/MyDrive/yolov5/results/detect/exp5/",
              "/content/drive/MyDrive/yolov5/results/detect/exp6/"]

out_path = src_images
for i in range(5, n_dims):
  applyGroundTruth(src_annotations[i], src_images[i], out_path[i])
  